# Abstract Factory
> Provide an interface for creating families of related or dependent objects without specifying their concrete classes.

## Problem
Lets say I have a family of objects that go together. In the true and tried example, lets say I have GUI widget library for web and QT. Moreover, the way these are instantiated are different for each widget. In the canvas code below, it has to know how to instantiate all types of widgets, even though it will not need to use some of that knowledge anywhere else. E.g., it needs to specify the exact size of the QT button, but it does not need this information anywhere else. How can I decopule the canvas from having to know these things?

In [5]:
from abc import ABC, abstractmethod

class Button(ABC):
    @property
    @abstractmethod
    def caption(self) -> str:
        raise NotImplemented()

    @abstractmethod
    def render(self) -> None:
        raise NotImplemented()

class TextBox(ABC):
    @property
    @abstractmethod
    def contents(self) -> str:
        raise NotImplemented()

    @abstractmethod
    def render(self) -> None:
        raise NotImplemented()

# QT family of widgets
class QtButton(Button):
    def __init__(self, caption: str, height: int, width: int) -> None:
        self._caption = caption
        self._height = height
        self._width = width

    @property
    def caption(self) -> str:
        return self._caption

    def render(self) -> None:
        print(f'Rendering a button {self._height} x {self._width}')

class QtTextBox(TextBox):
    def __init__(self, contents: str, height: int, width: int):
        self._contents = contents
        self._height = height
        self._width = width

    @property
    def contents(self) -> str:
        return self._contents

    def render(self) -> None:
        print(f'Rendering text box {self._height} x {self._width}')

# Web family of widgetes
class WebButton(Button):
    def __init__(self, caption: str) -> None:
        self._caption = caption

    @property
    def caption(self) -> str:
        return self._caption

    def render(self) -> None:
        print('Rendering a standard web button')


class WebTextBox(TextBox):
    def __init__(self, numrows: int, contents: str):
        self._numrows = numrows
        self._contents = contents

    @property
    def contents(self) -> str:
        return self._contents

    def render(self) -> None:
        print(f'Rendering textbox with {self._numrows} rows')

In [3]:
import random

def canvas():
    # determine the environment at runtime
    env = random.choice(["web", "qt"])
    if env == "web":
        print("Creating up web widgets")
        button = WebButton(caption="Submit")
        textbox = WebTextBox(numrows=3, contents="Enter your essay here")
    elif env == "qt":
        print("Creating QT widgets")
        button = QtButton(caption="Ok", height=3, width=10)
        textbox = QtTextBox(contents="Enter your essay here", height=10, width=20)
    # Now do some fancy layout stuff that canvas object usually do
    button.render()
    textbox.render()

canvas()

Creating up web widgets
Rendering a standard web button
Rendering textbox with 3 rows


## Solution

### OO Solution
Abstract the constructor calling in another **factory** class, with each family of objects having their own factory. Further abstract the factory interface out so the canvas can be given the factory interface which it will use to create widgets without knowing or caring what kind of environment it is running in.

In [6]:
class WidgetFactory(ABC):
    @abstractmethod
    def create_button(self) -> Button:
        pass

    @abstractmethod
    def create_textbox(self) -> TextBox:
        pass

class WebWidgetFactory(WidgetFactory):
    def create_button(self) -> Button:
        return WebButton(caption="Submit")

    def create_textbox(self) -> TextBox:
        return WebTextBox(numrows=3, contents="Enter your essay here")

class QtWidgetFactory(WidgetFactory):
    def create_button(self) -> Button:
        return QtButton(caption="Ok", height=3, width=10)

    def create_textbox(self) -> TextBox:
        return QtTextBox(contents="Enter your essay here", height=10, width=20)

In [8]:
import random

def canvas(widget_factory: WidgetFactory):
    button = widget_factory.create_button()
    textbox = widget_factory.create_textbox()
    # Now do some fancy layout stuff that canvas object usually do
    button.render()
    textbox.render()

# Bootstrapper
env = random.choice(["web", "qt"])
if env == "web":
    canvas(WebWidgetFactory())
elif env == "qt":
    canvas(QtWidgetFactory())

Rendering a standard web button
Rendering a standard web button


### Pythonic Solution
Instead of creating a new factory heirarchy which will pretty much mirror the original object heirarchy, pass the ctor as partial functions to the canvas. If there are too many widgets all the "factory" functions can be packaged together in a single `widget_makers` dict.

In [9]:
import functools

def canvas(button_maker, textbox_maker):
    button = button_maker()
    textbox = textbox_maker()
    # Now do some fancy layout stuff that canvas object usually do
    button.render()
    textbox.render()

# Bootstrapper
env = random.choice(["web", "qt"])
if env == "web":
    button_maker = functools.partial(WebButton, caption="Submit")
    textbox_maker = functools.partial(WebTextBox, numrows=3, contents="Enter essay here")
elif env == "qt":
    button_maker = functools.partial(QtButton, caption="Ok", height=3, width=10)
    textbox_maker = functools.partial(QtTextBox, contents="Enter your essay here", height=10, width=20)
canvas(button_maker, textbox_maker)

Rendering a standard web button
Rendering textbox with 3 rows
